1\. Write a function that converts number representation (bin<->dec<->hex)

In [19]:
def convert(string):
    
    no_hex = hex(string)
    no_bin = bin(string)
    
    if type(string) == int:
        print("{} equals ".format(string), no_bin, "in binary and ", no_hex, "in hexadecimal")
        
    elif type(string) == str and string[1] == "x":        #this means it's hex.
        
        decimal = int(string,16)
        print("{} equals ".format(no_hex), bin(decimal), "in binary and ", decimal, "in decimal")
    
    #elif type(string) == str and string[1] == 0 or string[1] == 1:
        
string = 0b1010
# integer = int(string,16)
# print(integer)
#convert(string)
type(string)
# string[1]

int

2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

In [1]:
# the first three words were generated by random typing of 0's and 1's. The fourth I generated from
# https://www.h-schmidt.net/FloatConverter/IEEE754.html to make sure I got the right value.
# the list entry words[3] is equal to 15.

words = ["10100101011101001000101000101110","10010101011101001000100000101110","10010101011101001000100000000000", "01000001011100000000000000000000"]

def flotation_device(word):
    sign = int(word[0])
    
    exponent_bin = word[1:9]
    exponent = int(exponent_bin,2)
    
    mantissa_bin_flipped = word[9:32]
# since the highest bit corresponds to the smallest number, we flip the order to iterate over the string later.
    mantissa_bin = mantissa_bin_flipped[::-1]
    
    mantissa = 1
    
    for i in range(len(mantissa_bin)):
        mantissa += int(mantissa_bin[len(mantissa_bin)-1-i])*2**(-1-i)
    
    result = ((-1)**sign)*mantissa*2**(exponent-127)
    
    print("{a} corresponds to the float {b}".format(a = word,b = result))
    
    return result

for i in range(len(words)):
    flotation_device(words[i])

10100101011101001000101000101110 corresponds to the float -2.1210443506098568e-16
10010101011101001000100000101110 corresponds to the float -4.9382834415279246e-26
10010101011101001000100000000000 corresponds to the float -4.938269266683534e-26
01000001011100000000000000000000 corresponds to the float 15.0


3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

In [33]:
import numpy as np
import sys

underflow_test = 1
overflow_test = float(1)
i = 0
counter = 0

# underflow limit test
while i < 1:
    test = underflow_test*0.5
    
    if test == 0.0:
        break
    underflow_test = test #updates the variable underflow_test only if underflow_test_new hasn't underflown.
    
print(underflow_test)

#overflow limit test
while i < 1:
    test = overflow_test*2
    if test == (test+1):
        break
    overflow_test = test

print(test)
#At some point the integer exceeds 32 bits, so the interpreter will just allocate RAM for the number until its limit. Since my RAM is 8GB, it just runs endlessly, or until the RAM starts begging to interrupt the kernel. Setting the type of overflow_test to a float checks instead the float size.

# I do not understand why the limit test does not return the same value as sys.maxsize.

print(sys.maxsize)

5e-324
9007199254740992.0
9223372036854775807


In [18]:
type(overflow_test)

float

4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt{1-x^2} {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 
